# Functional API (Keras)
* sequntial API로는 복잡한 네트워크 구조를 만들기 어렵다. 
* 따라서 이를 위해 함수형 API를 사용한다.
* 이러한 네트워크 구조의 예시로는 Wide Deep NN이 있다.

In [2]:
#캘리포니아 주택 가격 데이터 셋을 통해 예시를 살펴본다.
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full , X_test , y_train_full , y_test = train_test_split(
    housing.data, housing.target
)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full , y_train_full
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [3]:
import tensorflow as tf
from tensorflow import keras

input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation="relu")(input_)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.Concatenate()([input_, hidden2])
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs=[input_],outputs=[output])

# 모델 생성
* 캘리포니아 주택 가격 문제는 회귀 문제이므로 output 층의 뉴럴 개수가 1개이다. 이 곳에서 가격 예측을 해서 딱 하나로 나오게 된다.
* input 층에서는 모델의 입력을 정의한다.
* hidden1의 경우 입력을 그대로 전달 받고 있다. 이처럼 어떠한 층에 대해 입력을 뒤쪽의 인자처럼 받아 함수형 api 라고 한다.
* hidden2는 hidden1의 결과를 입력으로 전달 받는다.
* concatenate는 사슬 같이 잇는다 라는 뜻으로 input층과 hidden2를 입력받는다.
* 이러한 과정을 통해 최종 output은 1개의 뉴럴을 가진 층으로 나오고 이를 통해 사용할 입력과 출력 층을 만들어 Model을 만든다.

# 구조
input -> hidden1 -> hidden2 -> concat -> output<br>
intput - - - - - - - - - - - - - - - - - - -> concat

<br><br>
이후 모델을 훈련 , 평가 , 검증 , test 하는 과정은 다른 경우과 같게 일반적인 방식으로 구현된다.

이외에 모델을 만드는 방식에서 feature를 나누어 입력층을 다수 만드려면 다음과 같이도 만들 수 있다.

In [4]:
input_a = keras.layers.Input(shape=[5],name="wide_input")
input_b = keras.layers.Input(shape=[6],name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_b)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.Concatenate()([input_a, hidden2])
output = keras.layers.Dense(1, name="output")(concat)
model = keras.Model(inputs=[input_a,input_b],outputs=[output])

다음과 같은 모델의 경우 구조는 다음과 같을 것이다.

<br><br>
input_a - - - - - - - - - - - - - - - - - - - > concat => output<br>
input_b -> hidden1 -> hidden2 -> concat => output

# 모델 구조
* 이전과 다른점은 input이 2개로 나뉘었기 때문에 모델의 fit()을 진행할 때 , 2개의 input이 들어가야 한다는 점이다. 이때 튜플형태로 입력이 들어간다.
* 혹은 튜플이 아닌 이름과 값 자체를 mapping 해 딕셔너리 형태로 입력해도 된다.
* 이는 valid 데이터나 evaluate() , predict() 에도 동일하다.

In [5]:
model.compile(loss="mse",optimizer=keras.optimizers.SGD(lr=1e-3))

X_train_a , X_train_b = X_train[:, :5],X_train[:,2:]
X_valid_a , X_valid_b = X_valid[:, :5],X_valid[:,2:]
X_test_a , X_test_b = X_test[:, :5],X_test[:,2:]
X_new_a , X_new_b = X_test_a[:3], X_test_b[:3]

history = model.fit((X_train_a,X_train_b), y_train , epochs=20,
                    validation_data=((X_valid_a,X_valid_b), y_valid))
mse_test = model.evaluate((X_test_a,X_test_b) , y_test)
y_pred = model.predict((X_new_a,X_new_b))

#위의 코드에서 볼 수 있듯이 , 2개로 나누어진 입력에 대해서
#fit , evalutae , predict 모두 2개로 이루어진 튜플로 입력받고 있다.

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/20
363/363 [==============================] - 1s 2ms/step - loss: 2.0211 - val_loss: 0.9457
Epoch 2/20
363/363 [==============================] - 1s 2ms/step - loss: 0.8167 - val_loss: 0.7193
Epoch 3/20
363/363 [==============================] - 1s 2ms/step - loss: 0.6834 - val_loss: 0.6501
Epoch 4/20
363/363 [==============================] - 1s 2ms/step - loss: 0.6291 - val_loss: 0.6134
Epoch 5/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5939 - val_loss: 0.5856
Epoch 6/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5682 - val_loss: 0.5634
Epoch 7/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5466 - val_loss: 0.5471
Epoch 8/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5298 - val_loss: 0.5307
Epoch 9/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5154 - val_loss: 0.5195
Epoch 10/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5034 - val_loss: 0.5075

# 다중 출력이 요구되는 경우
* 같은 데이터 셋을 이용하지만 다른 작업의 결과를 요구하는 경우 , 예를 들어 같은 데이터 셋을 가지고 회귀 문제와 분류 문제를 만드는 경우 이 두 결과는 서로 다르지만 같은 데이터 셋을 공유한다.
* 두개 이상의 결과가 상호 보완적인 경우 , 예를 들어 물건의 좌표 (회귀)와 물건의 종류 (분류) 처럼 서로의 결과가 상호 보완적인 경우이다.
* 규제도구로써도 사용가능하다. 즉 하위 네트워크가 다른 네트워크에 의존하는지 의존하지 않는지 확인하고 , 그 자체로 유용한지 확인할 수 있다.


In [6]:
#보조출력 추가
input_a = keras.layers.Input(shape=[5],name="wide_input")
input_b = keras.layers.Input(shape=[6],name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_b)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.Concatenate()([input_a, hidden2])

output = keras.layers.Dense(1, name="main_output")(concat)
aux_output = keras.layers.Dense(1, name="aux_output")(hidden2)
model = keras.Model(inputs=[input_a,input_b],outputs=[output,aux_output])

#위의 코드와 같이 적절하게 output 층을 추가하는 것으로 쉽게 다중 출력을 구현할 수 있다.

* 각 출력은 각자의 loss function이 필요하다. 
* 따라서 이를 모델 컴파일 메소드에 전달해야한다.
* 정확히 하자면 loss function의 리스트를 전달해야한다.
* 각 출력의 loss에 대해 중요도가 다를 수 있으므로 이에 더해 주 출력과 보조출력으로 나누어 각자에 대해 weight를 적용할 수도 있다.
* keras는 모든 loss 값을 더해 model training에 사용하기 떄문이다.

In [7]:
model.compile(loss=["mse","mse"],loss_weights=[0.9,0.1], optimizer="sgd")

#이처럼 각각의 출력에 대해 loss function list를 전달하고 이에 대해
#가중치를 두어 중요도를 달리 적용할 수 있다.

In [9]:
history = model.fit([X_train_a,X_train_b],[y_train,y_train], epochs=20,
                    validation_data=([X_valid_a,X_valid_b], [y_valid, y_valid])
                    )


Epoch 1/20
363/363 [==============================] - 2s 3ms/step - loss: 0.9955 - main_output_loss: 0.8358 - aux_output_loss: 2.4326 - val_loss: 0.5942 - val_main_output_loss: 0.5227 - val_aux_output_loss: 1.2377
Epoch 2/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5485 - main_output_loss: 0.4836 - aux_output_loss: 1.1326 - val_loss: 0.5185 - val_main_output_loss: 0.4626 - val_aux_output_loss: 1.0219
Epoch 3/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4967 - main_output_loss: 0.4460 - aux_output_loss: 0.9527 - val_loss: 0.4865 - val_main_output_loss: 0.4434 - val_aux_output_loss: 0.8747
Epoch 4/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4638 - main_output_loss: 0.4236 - aux_output_loss: 0.8260 - val_loss: 0.4562 - val_main_output_loss: 0.4215 - val_aux_output_loss: 0.7686
Epoch 5/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4563 - main_output_loss: 0.4255 - aux_output_loss: 0.7334 - val_loss

* 위의 경우 주 출력과 보조출력 모두 같은 것을 예측하므로 동일한 label을 사용한다.
* 만약 예측해야하는 것이 다르다면 다른 y , 즉 label을 각각 자리에 맞게 넣어주면 된다.
* 또한 training 과정과 validation , test에 있어서도 출력이 2개가 나오게 될 것이므로 위와 같이 각자 자리에 맞는 데이터를 넣어주면 된다.


In [10]:
total_loss , main_loss , aux_loss = model.evaluate(
    [X_test_a,X_test_b],[y_test,y_test]
)
#전체 loss값과 main , aux에 대한 loss값을 따로 구할 수 있다.
#즉 evaluate 단계에서 개별손실과 총손실 모두 return 한다는 것이다.

y_pred_main , y_pred_aux = model.predict([X_new_a,X_new_b])
#prediction의 경우도 각각 주 출력과 보조출력에 대하여 각각 예측한다.

162/162 [==============================] - 0s 1ms/step - loss: 0.3498 - main_output_loss: 0.3361 - aux_output_loss: 0.4729
